In [ ]:
from model.utils import get_token_embeddings_table, SMALL_MODEL, LARGER_MODEL
get_token_embeddings_table(LARGER_MODEL, "output").shape


In [1]:

import json
with open("data/most_similar_global_input.json", "r") as f:
    data = json.load(f)

data.keys()

dict_keys(['small', 'large'])

In [10]:
data["large"]["0"]["0"].keys()

dict_keys(['tokens', 'similarities'])

In [ ]:
import json
with open("data/prompt_similarities.json", "r") as f:
    data = json.load(f)

data.keys()

In [ ]:
len(data["large"]["layers"].keys())


In [ ]:
len(data["large"]["prompt_tokens"])

In [ ]:
len(data["small"]["layers"])

In [ ]:

import numpy as np
from model.utils import cosine_similarity

emb = np.arange(1000).reshape(1, 1000)
embeddings = np.arange(100000).reshape(1000, 100)

def similarity_except_max(emb, embeddings):
    prompt_similarities = cosine_similarity(emb, embeddings)
    max_sim = np.max(prompt_similarities, axis = 1)
    mask = prompt_similarities<max_sim

    second_to_max = np.max(np.where(~mask, -1e9, prompt_similarities), axis = 1)
    clipped = np.clip(prompt_similarities, np.min(prompt_similarities, axis = 1), second_to_max)

    normalized = (clipped - np.min(prompt_similarities, axis = 1)) / (second_to_max - np.min(prompt_similarities, axis = 1))
    return normalized



In [ ]:
similarity_except_max(emb, embeddings)